In [17]:
import pandas as pd
import numpy as np
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.backends.backend_tkagg as tkagg
from matplotlib.figure import Figure
from scipy import stats
import os
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, scale
import scipy.signal as signal
import shutil
import seaborn as sns
import statistics
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import *
import re
from tkinter import ttk
from tkinter import filedialog
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
import timeit
from scipy.signal import butter, lfilter
from scipy.signal import freqz
import scipy
from moviepy.video.io.ffmpeg_tools import ffmpeg_merge_video_audio

plt.rcParams['figure.dpi'] = 100
sns.set_theme(context='paper', style='ticks')
sns.set_context("paper", rc={"font.size":5,"axes.titlesize":10,"axes.labelsize":1})

In [18]:
def dialog_box():

    #Create an instance of Tkinter frame
    win= tk.Tk()
    win.title('PSD and Coherence extractor')
    frame1 = tk.Frame(win)
    frame1.pack(padx=1,pady=1)
    
    #Define the geometry
    win.geometry("350x500")
    win.configure(background='white')
    def select_blocks_folder():
        global all_files_raw_path
        all_files_raw_path = filedialog.askdirectory()
        tk.Label(win, text=all_files_raw_path, font=13,bg="white").pack()
        
    def select_videos_folder():
        global movies_folder
        movies_folder = filedialog.askdirectory()
        tk.Label(win, text=movies_folder, font=13,bg="white").pack()

    # Select path to folder with extracted blocks txt files
    tk.Label(win, text="Select blocks folder", font=('Aerial 10 bold'),bg="white").pack(pady=5)
    button_blocks= ttk.Button(win, text="Select", command= select_blocks_folder)
    button_blocks.pack(ipadx=5, pady=15)
    # Select path to folder with videos from ECoG recording .WMV extension
    tk.Label(win, text="Select videos folder", font=('Aerial 10 bold'),bg="white").pack(pady=5)
    button_videos= ttk.Button(win, text="Select", command= select_videos_folder)
    button_videos.pack(ipadx=5, pady=15)
    # fill desired length of epoch, usually 5 seconds
    tk.Label(win, text="Epochs desired length in seconds", font=('Aerial 10 bold'),bg="white").pack(pady=5)
    epoch_length_input = tk.Entry(win)
    epoch_length_input.pack()
    # multiply EMG TKEO harmonic mean by 2.
    #If you are not getting enough epochs, increase this threshold.
    #if you're getting too many epochs with movement - decrease this multiplier.
    #tk.Label(win, text="Multiply EMG threshold by", font=('Aerial 10 bold'),bg="white").pack(pady=5)
    #threshold_multiplication = tk.Entry(win)
    #threshold_multiplication.pack()
    #insert number of drugs (or blocks) recorded, if none - input 0.
    tk.Label(win, text="How many drugs?", font=('Aerial 10 bold'),bg="white").pack(pady=5)
    num_drugs = tk.Entry(win)
    num_drugs.pack()
    
    
    
    
    # write genotype
    tk.Label(win, text="Write genotype", font=('Aerial 10 bold'),bg="white").pack(pady=5)
    genotype_input = tk.Entry(win)
    genotype_input.pack()
    
    def getInput():
        user_input = genotype_input.get()
        ep_input = epoch_length_input.get()
        #EMG_multiply_input = threshold_multiplication.get()
        number_of_drugs = num_drugs.get()
        win.destroy()
        global genotype
        global ep_len
        #global emg_multiplication
        global drugs_num
        genotype = user_input
        ep_len = ep_input
        #emg_multiplication = EMG_multiply_input
        drugs_num = number_of_drugs
    
    tk.Button(win, text = "submit",
               command = getInput).pack(ipadx=5, pady=15)
    
    win.mainloop()  

In [19]:
def get_emg(data,fs):

    emg =  data.EMG.copy()
    tkeo = (emg.iloc[1:-1]**2 ).values - (emg.iloc[0:-2]*emg.iloc[2:].values)
    tkeo = np.append(0,tkeo)
    tkeo = np.append(tkeo,0)
    
    emg_rectified = abs(tkeo)
    low_pass = 3/(fs/2)
    b2, a2 = signal.butter(4, low_pass, btype='lowpass')
    emg_envelope = signal.filtfilt(b2, a2, emg_rectified)
    # tkeo = ( tkeo-np.mean(tkeo) )/ np.std(tkeo)
    # tkeo = stats.zscore(emg)
    data['TKEO'] = emg_envelope/emg_envelope.max()
    return data

In [20]:

def get_PSD(data,basename,dirname,emg_threshold,fs,sec):

    def create_ep_matrix(data,fs):    
        
        def reshape_matrix(x,n,ep):
            def window_stack(a, stepsize=1, width=2):
                n = a.shape[0]
                return np.hstack( a[i:1+n+i-width:stepsize] for i in range(0,width) )                   
            R_reshape=pd.DataFrame(np.array(x).reshape(int(n/ep),-1))
            R_overlap=pd.DataFrame(window_stack(R_reshape, stepsize=1, width=2))
            windows =R_overlap.T  
            return windows
 
        ep = sec/2*fs
        n= data.shape[0]
        w_overlap =int(ep) #2.5 sec sliding window
        x = data.index.copy()
        remove = n%w_overlap
        if remove !=0:
            x = x[:-(remove)]
            n = n - remove
            windows = reshape_matrix(x,n,ep)
            
        else:
            windows = reshape_matrix(x,n,ep)
        epochs = {}
        for col in windows.columns:
            epochs[col] = data.loc[windows.iloc[:,col]]
        return epochs
   
    epochs = create_ep_matrix(data,fs)
   
    epochs_psd_eeg = dict()
    for key, val in epochs.items():
        f,PxxL = signal.welch(
            val.SomL_Cer,fs=1000,  window='hann',nperseg = 1024
        )
        
        f, PxxR = signal.welch(
            val.SomR_Cer,fs=1000,  window='hann',nperseg = 1024
        )
        
        Av_Pxx= np.mean( np.array([ PxxL, PxxR ]), axis=0 )
        
        f,emg_p = signal.welch(
            val.EMG,fs=1000,  window='hann',nperseg = 1024
        )
        epochs_psd_eeg[key] = pd.DataFrame({'PSD': Av_Pxx, 'EMG_power':emg_p}, columns=['PSD', 'EMG_power'], index = f)
    delta_power = []
    theta_power = []
    alpha_power = []
    beta_power = []
    gamma_power = []
    total_power = []
    emg_power = []
    for key, val in epochs_psd_eeg.items():
       delta_power.append(val.PSD[1:5].sum())
       theta_power.append(val.PSD[5:9].sum())
       alpha_power.append(val.PSD[9:13].sum())
       beta_power.append(val.PSD[13:31].sum())
       gamma_power.append(val.PSD[31:103].sum())
       total_power.append(val.PSD[1:103].sum())
       emg_power.append(val.EMG_power[21:307].sum())
       
       
             
    spectral_params = pd.DataFrame(
        list(zip(delta_power,theta_power,alpha_power,beta_power,gamma_power,total_power,emg_power)),
        columns =['Delta power','Theta power','Alpha power','Beta power','Gamma power','Total power','EMG power']
        ,index = epochs_psd_eeg.keys()
        )
    spectral_params['Theta/Delta ratio'] = spectral_params['Theta power']/spectral_params['Delta power']
        

    delta_thresh = spectral_params['Delta power'].quantile(0.5)
    ratio_thresh =spectral_params['Theta/Delta ratio'].quantile(0.75) 
    
    
    quiet_awake_epochs_index = spectral_params.index[
        (spectral_params['Delta power']< delta_thresh) & 
        (spectral_params['Theta/Delta ratio']<ratio_thresh) ].tolist() 

    quiet_awake_epochs={}
    for key in quiet_awake_epochs_index:
        if epochs[key].TKEO.max() <emg_threshold:
            quiet_awake_epochs[key]= epochs[key]
        
        
    cal=False
    if len(list(quiet_awake_epochs.keys()))<1 : 
        print('calculating new emg threshold')
        cal = True
 
    count = 1
    th=0
    while len(list(quiet_awake_epochs.keys())) < 1 :
        quiet_awake_epochs={}
        count +=0.5
        for key in quiet_awake_epochs_index:
            th = emg_threshold*count 
            if epochs[key].TKEO.max()<= th:
                quiet_awake_epochs[key]= epochs[key]
        
    if cal == True: print('New EMG threshold: ',th )

    quiet_awake_epochs_index = list(quiet_awake_epochs.keys())
    
    quiet_awake_epochs_index_times = [t.index[0] for k,t in quiet_awake_epochs.items()]

    
    colors = ['b', 'g', 'c', 'm']
    thresholds = [delta_thresh,ratio_thresh] 
    columns_fig = ['Delta power','Theta/Delta ratio']

    fig, axes = plt.subplots(1, 3);
    plt.subplots_adjust(hspace=0.5)
    fig.suptitle(basename + ' thresholds')
    axes = axes.ravel()
    for i in range(0,2):
        threshold =thresholds[i]
        sns.scatterplot(ax =axes[i], data= spectral_params[columns_fig[i]],
                         color=colors[i],s=1.5)
        axes[i].axhline(threshold,color='k', ls='--',lw=1)
        axes[i].set_title(columns_fig[i])
    sns.scatterplot(ax =axes[2], data=
                    spectral_params['EMG power']
                    ,color=colors[3],s=1.5)
    axes[2].set_title('EMG power')
    figures_path = os.path.join(dirname, 'figs\\')
   

    fig.tight_layout()
    plt.show()
    if os.path.exists(figures_path):
        fig.savefig(figures_path +basename+' spectrals.png', bbox_inches='tight')
    else: 
        os.mkdir(figures_path)
        fig.savefig(figures_path +basename+' spectrals.png', bbox_inches='tight')



    
    PSD_vals = spectral_params.iloc[quiet_awake_epochs_index]
    PSD_vals['delta_relative'] = PSD_vals['Delta power']/ PSD_vals['Total power']*100
    PSD_vals['theta_relative'] = PSD_vals['Theta power']/ PSD_vals['Total power']*100
    PSD_vals['alpha_relative'] = PSD_vals['Alpha power']/ PSD_vals['Total power']*100
    PSD_vals['beta_relative'] = PSD_vals['Beta power']/ PSD_vals['Total power']*100
    PSD_vals['gamma_relative'] = PSD_vals['Gamma power']/ PSD_vals['Total power']*100
    PSD_vals['Total power uV'] = PSD_vals['Total power']*10**6
    
    Averaged_spectral_params = pd.DataFrame({basename: PSD_vals.mean(axis=0)})
    Averaged_PSD =pd.DataFrame({ basename: pd.concat([epochs_psd_eeg[key].PSD for key in quiet_awake_epochs_index], axis=1).mean(axis=1)})
    number_of_epochs = pd.DataFrame({basename: [len(quiet_awake_epochs_index)]})
    individual_epochs_psdvals =  {basename: PSD_vals}
    all_epochs_psd = pd.concat([epochs_psd_eeg[key].PSD for key in quiet_awake_epochs_index], axis=1,keys =quiet_awake_epochs_index_times )
    all_epochs_psd_V = {basename: all_epochs_psd}
    
    final_spect = {}
    for key, val in quiet_awake_epochs.items():
        f, t, Zxx = signal.stft(
            val.SomL_Cer, fs=1000,  window='hann',nperseg = 1024
        )
        final_spect[key] = np.abs(Zxx)**2
    spectogram_all_ep = np.concatenate([final_spect[x] for x in final_spect], 1)
    if len(np.arange(0, ((t[1]-t[0])*spectogram_all_ep.shape[1]) , t[1]-t[0])) == spectogram_all_ep.shape[1]:
        time = np.arange(0, ((t[1]-t[0])*spectogram_all_ep.shape[1]) , t[1]-t[0])
    else:
        time = np.arange(0, (((t[1]-t[0])*spectogram_all_ep.shape[1])-(t[1]-t[0])) , t[1]-t[0])
    z = spectogram_all_ep[1:103,:]
    levels = MaxNLocator(nbins=256).tick_values(z.min(), z.max())
    cmap = plt.colormaps['jet']
    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
    
    fig, (ax0, ax1,ax2,ax3) = plt.subplots(nrows=4);
    im =ax0.pcolormesh(time, f[:100], z[:100,:],cmap =cmap ,norm = norm, shading = 'gouraud')
    ax0.set_title(basename+' spectrogram')
    fig.colorbar(im, ax=ax0)
    sig = pd.concat([x['SomL_Cer'] for x in quiet_awake_epochs.values()],axis=0).reset_index()['SomL_Cer']
    sig_time = np.array(sig.index)
    sig_plt = ax1.plot(sig_time, sig) 
    ax1.set_title('signal')
    emg_sig = pd.concat([x['TKEO'] for x in quiet_awake_epochs.values()],axis=0).reset_index()['TKEO']
    sig_emg_plt = ax2.plot(sig_time, emg_sig) 
    ax2.set_title('emg TKEO')
    ax3.set_title('TKEO of entire block')
    ax3.plot(data['TKEO']) 
    ax3.set_ylim(-0.1,0.6)
    fig.tight_layout()
    plt.show()
    fig.savefig(figures_path +basename+' epochs.png', bbox_inches='tight')
    

                
    return individual_epochs_psdvals, Averaged_spectral_params,quiet_awake_epochs,Averaged_PSD,number_of_epochs,all_epochs_psd_V


In [21]:
def make_movie_QA(movie_file,final_epochs,final_movie_name):
    if Path(movie_file).is_file():
        all_movies = os.listdir(os.path.dirname(movie_file))
        all_movies_count = len([f for f in all_movies if not f.endswith('i')])
        clip = VideoFileClip(movie_file)
        duration = clip.duration
        if duration<3600 and all_movies_count>2:
            movie_file = movie_file[:-5] + str(int(movie_file[-5])+1) + movie_file[-4:]
            print('epoch length is too short, taking next video')
            clip = VideoFileClip(movie_file)
            duration = clip.duration       
        else:
            clip = VideoFileClip(movie_file)
            duration = clip.duration
            
        subs = []
        for key,val in final_epochs.items(): 
            if duration >val.index[0]:
                sub =  clip.subclip(val.index[0] ,val.index[-1] )
                subs.append(sub)
        if duration >val.index[0]:
            final_clip = concatenate_videoclips(subs)
            # final_clip = final_clip.fx( vfx.speedx, 4)
            final_clip.write_videofile(final_movie_name,codec = 'libvpx',threads = 150, fps=24, preset='ultrafast')
            print(final_movie_name+' extracted!') 
        else: print("can't generate " +final_movie_name+" movie. original is not long enough")
    else: print(final_movie_name + ' original video not found')

In [22]:
def get_coherence(files):
    output_COH=dict()
    output_IMCOH = dict()
    output_MSC = dict()
    for segment in files:
        df = pd.read_csv(segment, sep="\t", index_col = 0,names=["SomL_Cer", "SomR_Cer"]).dropna()   
        # calculate spectral density on signal x and y
        fs = 1000 #sampling frequency of the x and y    
        x = df.SomL_Cer
        y = df.SomR_Cer
        freq, Cxx = signal.csd(x, x, fs, window='hann',nperseg = 1024)
        freq, Cyy = signal.csd(y, y, fs, window='hann',nperseg = 1024)
        freq, Cxy = signal.csd(x, y, fs, window='hann',nperseg = 1024)
        # coherence
        COH = np.abs(Cxy) / np.sqrt(Cxx * Cyy)
        # magnitude squared coherence
        MSC = np.abs(Cxy) ** 2 / (Cxx * Cyy)
        # coherence from scipy (MSC)
        #fr, COH_SCIPY = signal.coherence(x, y, fs, window="hann",nperseg = 1024)
        # imaginary part of coherency
        IMCOH = Cxy.imag / np.sqrt(Cxx * Cyy)
        #results = {"frequency":freq, "msc_scipy": COH_SCIPY, "coh": COH, "msc":MSC, "imcoh": IMCOH}
        results = {"frequency":freq, "coh": COH,"imcoh": IMCOH,"msc":MSC }
        df_results = pd.DataFrame.from_dict(results).set_index('frequency')
        output_COH[str(segment.stem)] = df_results.coh
        output_IMCOH[str(segment.stem)] = df_results.imcoh     
        output_MSC[str(segment.stem)] = df_results.msc     
    return output_COH,output_IMCOH,output_MSC

In [23]:

def get_excels(all_files_raw):
    QA_coherence = []
    QA_imCOH = []
    QA_MSC = []
    PSD = []
    spectral_averages = []
    num_of_epochs = []
    epoch_spectrals  = []
    epochs_PSD = []
    
    for file in all_files_raw:
        os.chdir(all_files_raw_path)
        dirname = os.path.dirname(file)
        basename = os.path.basename(file)[:-4].rstrip()   
        QA_path = os.path.join(dirname, basename,'QA')
        if os.path.exists(os.path.join(dirname, basename)):
            shutil.rmtree(os.path.join(dirname, basename))
        os.mkdir(os.path.join(dirname, basename))
        if os.path.exists(QA_path):
            shutil.rmtree(QA_path)
    
        os.mkdir(QA_path)
        data = pd.read_csv(file, sep="\t", index_col=0, names=["SomL_Cer", "SomR_Cer","EMG"]).dropna()  
        fs = 1000 #sampling frequency of the x and y   
        data = get_emg(data,fs)
        TKEO = data['TKEO']
        class App:
            def __init__(self, master):
                # Create a container
                frame = tk.Frame(master)
                # Create 2 buttons

                self.button_left = tk.Button(frame,text="< Decrease threshold",
                                                command=self.decrease)
                self.button_left.pack(side="left")
                self.button_right = tk.Button(frame,text="Increase threshold >",
                                                command=self.increase)
                self.button_right.pack(side="left")

                self.threshold= tk.Button(frame,text="print threshold", command= self.print_threshold)
                self.threshold.pack(side="bottom")


                fig = Figure(figsize=(4,2))
                ax = fig.add_subplot(111)
                self.emg = ax.plot(data['TKEO'])
                self.line, = ax.plot(
                    data['TKEO'].index,np.ones(len(data['TKEO']))*(data['TKEO'].quantile(0.25)+data['TKEO'].std()/2),#)+data['TKEO'].std()*2
                    linewidth=0.4)
                # ax.set_ylim([-data['TKEO'].max()*0.25,data['TKEO'].max()*0.25])
                ax.set_xlim([data['TKEO'].index[0],data['TKEO'].index[-1]])
                self.canvas = FigureCanvasTkAgg(fig,master=master)
                self.canvas.draw()
                self.canvas.get_tk_widget().pack(side='top', fill='both', expand=1)
                fig.tight_layout()
                tkagg.NavigationToolbar2Tk(self.canvas, root)

                frame.pack()

            def decrease(self):
                x, y = self.line.get_data()
                self.line.set_ydata(y -data['TKEO'].std()*0.03)
                self.canvas.draw()

            def increase(self):
                x, y = self.line.get_data()
                self.line.set_ydata(y +data['TKEO'].std()*0.03)
                self.canvas.draw()

            def print_threshold(self):
                global emg_threshold
                emg_threshold = self.line.get_ydata().mean()
                print(self.line.get_ydata().mean())
                root.destroy()
        root = tk.Tk()    
        app = App(root)
        root.mainloop()
#         emg_threshold =scipy.stats.hmean(abs(TKEO[TKEO.between(TKEO.quantile(0),TKEO.quantile(0.6))]))*threshold_multiplication_emg
        #emg_threshold =scipy.stats.hmean(abs(TKEO))*2
        PSD_vals, Averaged_spectrals,quiet_awake_epochs,Averaged_PSD,number_of_epochs,all_epochs_psd =  get_PSD(data,basename,dirname,emg_threshold,fs,sec)
        spectral_averages.append(Averaged_spectrals),PSD.append(Averaged_PSD),num_of_epochs.append(number_of_epochs),
        epoch_spectrals.append(PSD_vals),epochs_PSD.append(all_epochs_psd)
        
        if any( re.findall('\d+', basename)[0] in  x[0] for x in os.walk(movies_folder)):
            movie_file =  os.path.join(Path([s for s in [x[0] for x in os.walk(movies_folder)]
                                    if re.findall('\d+', basename)[0]  in s][0]) ,
                              Path('Record'+re.findall('\d+', basename)[1]+'.wmv'))
            QA_movie =  make_movie_QA(movie_file,quiet_awake_epochs ,basename + ' QA.wmv')
        else: print(basename+' video not found')
        
    
        #extract all good epoches to txt files:
        os.chdir(QA_path)
        for key, val in quiet_awake_epochs.items():
          val[['SomL_Cer','SomR_Cer']].to_csv(str(key)+'.txt', sep='\t', encoding='utf-8',header = False)
        QA_epochs_path = Path(QA_path)
        coh_files = QA_epochs_path.glob('*.txt')
        coh, imcoh,MSC = get_coherence(coh_files)
        coh_df = pd.DataFrame.from_dict(coh, orient='columns')
        col_name = 'coherence_averaged_QA_' + basename
        coh_df[col_name] = coh_df.mean(axis=1)
        QA_coherence.append(coh_df[col_name])
        
        imcoh_df = pd.DataFrame.from_dict(imcoh, orient='columns')
        im_col_name = 'imaginary coherence_averaged_QA_' + basename
        imcoh_df[im_col_name] = imcoh_df.mean(axis=1)
        QA_imCOH.append(imcoh_df[im_col_name])
        
        MSC_df = pd.DataFrame.from_dict(MSC, orient='columns')
        msc_col_name = 'MSC coherence_averaged_QA_' + basename
        MSC_df[msc_col_name] = MSC_df.mean(axis=1)
        QA_MSC.append(MSC_df[msc_col_name])
        
        print( str(number_of_epochs)+' QA epochs')
        
    reports = {'coherence':  pd.concat(QA_coherence,axis=1).iloc[:103],
                'PSD':pd.concat(PSD,axis=1).iloc[:103],
                'spectral averages' : pd.concat(spectral_averages,axis=1),
                'imaginary coherence': pd.concat(QA_imCOH,axis=1).iloc[:103],
                'MSC coherence': pd.concat(QA_MSC,axis=1).iloc[:103],
                'number of epochs': pd.concat(num_of_epochs,axis=1)}

    return reports,dirname,epoch_spectrals,epochs_PSD

In [24]:
def save_excel_files(reports,dirname,epoch_spectrals,epochs_PSD,num_drugs):
    # saves excel files to a folder called excel files in the blocks folder
    os.chdir(dirname)
    if os.path.exists(os.path.join(dirname, 'excel files')):
        excel_path = os.path.join(dirname, 'excel files')
        os.chdir(excel_path)
    else:
        excel_path = os.path.join(dirname, 'excel files')
        os.mkdir(excel_path)
        os.chdir(excel_path)
        
    if int(num_drugs)>0:
        drugs_count = []
        for i in range(1,int(num_drugs)+2):
            drugs_count.append('block'+ str(i) )
    else: drugs_count = ['block1']

    for name in drugs_count:
        for key,val in reports.items():
            block_file =  pd.ExcelWriter( genotype + " " +key + name+ ".xlsx")
            val[[col for col in val.columns if name in col]].to_excel(block_file,index=True, engine='openpyxl')
            block_file.save()
        
        writer_block = pd.ExcelWriter(name+' individual_epochs_spectrals.xlsx', engine='openpyxl') 

        for x in epoch_spectrals: 
            for df_name, df in x.items():
                if name in df_name:
                    df.to_excel(writer_block, sheet_name=df_name)
        writer_block.save()
        
        writer_block_psd =  pd.ExcelWriter(name+' individual_epochs_PSDS.xlsx', engine='openpyxl') 
        
        for l in epochs_PSD: 
            for df_name, df in l.items():
                if name in df_name:
                    df.to_excel(writer_block_psd, sheet_name=df_name)
        writer_block_psd.save()

    print('done!')